g2a webscraper, v1
- feature: shows product, platform, product type, region, price, original price, site
- limitation: fails when sponsored links are present

In [16]:
#v1 g2a scraper

##############
# import modules
##############
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd

############
# get text and game name
# $future task: enable chrome headless mode, per https://stackoverflow.com/questions/46920243/how-to-configure-chromedriver-to-initiate-chrome-browser-in-headless-mode-throug/49582462#49582462
############
print('What game are you searching for? ')
game_title = input().lower()
# game_title = 'Baldur\'s Gate 3'.lower()
print(game_title)
print()
url_game_title = game_title.replace(' ', '%20').replace('\'', '%27').title()
url = 'https://www.g2a.com/search?query=' + url_game_title

driver = webdriver.Chrome() #bypass non-valid Chrome header by launching Chrome, per https://stackoverflow.com/questions/53918187/python-requests-to-continue-your-browser-has-to-accept-cookies-and-has-to-hav
driver.get(url)
soup = BeautifulSoup(driver.page_source,'lxml')
text = soup.get_text().lower()

# capture region lock, game system, product type, price
pattern = re.escape(game_title) + r"(.*?)" + re.escape('offer')
matches = re.findall(pattern, text)

# clean matches by price
price_clean = []
for i in matches:
    if 'out of stock' in i:
        oos_match = i.replace('out of stock', '`out of stock')
        price_clean.append(oos_match)
    if '$' in i:
        instock_match = i.replace('$', '`$')
        price_clean.append(instock_match)

# clean matches by platform
platform_clean = []
for i in price_clean:
    if 'platform' in i:
        platform_match = i.replace('platform', '`platform')
        platform_clean.append(platform_match)

# clean matches by type
type_clean = []
for i in platform_clean:
    if 'type' in i:
        type_match = i.replace('type', '`type')
        type_clean.append(type_match)

# clean matches by region
region_clean = []
for i in type_clean:
    if 'region' in i:
        region_match = i.replace('region', '`region')
        region_clean.append(region_match)

# clean matches by activation region
activation_region_clean = []
for i in region_clean:
    if 'can' in i:
        activation_region_match = i.replace('can', '`can')
        activation_region_clean.append(activation_region_match)

df = pd.DataFrame(activation_region_clean) #create dataframe
df1 = df[0].str.split('`', expand=True) #use '`' as column separator
df1.columns = ['product', 'platform', 'product type', 'region', 'activation region', 'price', 'original price'] # add column titles
df1 = df1.drop(columns=['activation region'])
df1 = df1.map(lambda x: x.strip() if isinstance(x, str) else x) # remove all leading and trailing spaces
df1['platform'] = df1['platform'].str.replace('platform: ', '').str.replace('account', '')
df1['product type'] = df1['product type'].str.replace('type:', '')
df1['region'] = df1['region'].str.replace('region:', '')
df1['price'] = df1['price'].str.replace('$ ', '')
df1['original price'] = df1['original price'].str.replace('$ ', '').str.replace(r'-.*', '', regex=True) # remove % discount
df1['site'] = 'G2A'

print('G2A\'s <<' + game_title + '>>')
g2a = df1.sort_values(by=['platform', 'price'])
display(g2a)

What game are you searching for? 
skyrim

G2A's <<skyrim>>


,product,platform,product type,region,price,original price,site
11,- pack steam key global,steam,key,global,10.23,42.84,G2A
4,- dawnguard (pc) - steam key - global,steam,key,global,10.98,21.42,G2A
2,- legendary edition steam key global,steam,key,global,11.71,42.84,G2A
0,anniversary upgrade (pc) - steam key - global,steam,key,global,16.52,21.42,G2A
9,anniversary edition (pc) - steam key - europe,steam,key,europe,22.57,53.56,G2A
10,hearthfire (pc) - steam key - global,steam,key,global,24.68,None,G2A
5,vr (pc) - steam key - europe,steam,key,europe,25.30,64.27,G2A
3,- dragonborn (pc) - steam key - global,steam,key,global,7.97,21.42,G2A
1,(pc) - steam key - global,steam,key,global,8.57,42.84,G2A
13,special edition + fallout 4 g.o.t.y bundle ste...,steam,key,global,out of stock,None,G2A
